In [1]:
import findspark
import time

from pyspark import SparkFiles
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [2]:
# Read the AWS S3 bucket into a DataFrame
url = 'https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv'
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

In [3]:
# Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView('home_sales')

In [26]:
# Calculate the average price for a four bedroom house sold per year, rounded to two decimal places
spark.sql('''
SELECT YEAR(date) AS sale_year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY Sale_Year
ORDER BY Sale_Year
''').show()

+---------+-------------+
|sale_year|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [28]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [34]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms, 
# with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

